In [0]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
!pip install facenet_pytorch
from facenet_pytorch import InceptionResnetV1
from torch.autograd import Variable
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.3.1
Torchvision Version:  0.4.2


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

training_dir = "/content/gdrive/My Drive/images_fer2013/Training"
testing_dir = "/content/gdrive/My Drive/images_fer2013/PublicTest"
validation_dir = "/content/gdrive/My Drive/images_fer2013/PrivateTest"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
model1 = InceptionResnetV1(pretrained='vggface2').eval()
device = torch.device("cuda" if torch.cuda.is_available()  else "cpu")
model1.to(device)
num_classes = 7

In [0]:
for param in model1.parameters(): 
    param.requires_grad = False

In [0]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Sequential(
            #nn.Linear(512, 1024),
            #nn.ReLU(inplace=True),
            
            #nn.Linear(1024, 512),
            #nn.ReLU(inplace=True),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(inplace=True),
            #nn.Linear(256, 128),
            #nn.BatchNorm1d(num_features=128),
            #nn.ReLU(inplace=True),

            nn.Linear(256, num_classes))
        self.dropout = nn.Dropout()
  
    def forward(self, x):
        x = self.fc1(x)
        return x

In [0]:
net = Model().cuda()


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), weight_decay=0.003, lr=0.009, amsgrad=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.8)
train_number_epochs = 100


In [0]:
transforms = transforms.Compose([transforms.Resize(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
                                       ])
train_data = datasets.ImageFolder(training_dir,       
                    transform=transforms)
train_loader = torch.utils.data.DataLoader(train_data,
                   shuffle=True, batch_size=224)
validation_data = datasets.ImageFolder(validation_dir,       
                    transform=transforms)
validation_loader = torch.utils.data.DataLoader(validation_data,
                   shuffle=True, batch_size=64)
test_data = datasets.ImageFolder(testing_dir,       
                    transform=transforms)
test_loader = torch.utils.data.DataLoader(test_data,
                   shuffle=True, batch_size=64)
print(train_loader.dataset.classes)

['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']


In [0]:
train_counter = []
train_loss_history = []
train_iteration_number= 0
train_accuracy_history=[]

valid_counter = []
valid_loss_history = []
valid_iteration_number= 0
valid_accuracy_history = []

valid_loss_min = np.Inf

In [0]:
#optimizer = optim.Adam(net.parameters(), lr=0.0003)
for epoch in range(0,train_number_epochs):
    
    train_class_correct = list(0 for i in range(num_classes))
    train_class_total = list(0 for i in range(num_classes))

    valid_class_correct = list(0 for i in range(num_classes))
    valid_class_total = list(0 for i in range(num_classes))
    running_val_accuracy = 0.0
    running_train_accuracy = 0.0
    running_loss = 0.0
    running_valid_loss = 0.0
    net.train()
    for i, data in enumerate(train_loader,0):
        
        img1 , label = data
        #img1, label = Variable(img1), Variable(label)
        img1 , label = img1.cuda(), label.cuda()
        #img1, label = Variable(img1), Variable(label)
        #row, img0, img1 , label = row.cuda(), img0.cuda(), img1.cuda() , label.cuda()
  
        optimizer.zero_grad()
        output1= model1(img1)      
        output = net(output1)
        _, pred= torch.max(output,1)
        
        #plt.imshow(img1.data[0])
        
        
        
        #a = plt.imshow(img1.data[0].permute(1, 2, 0))
        #plt.title(train_loader.dataset.classes[pred[0]])
        #a.set_title = train_loader.dataset.classes[pred[0]]
        #plt.show()

        loss = criterion(output,label)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        #print("----------------pred")
        #print(pred)
        #print("----------------label")
        #print(label)
        correct = pred.eq(label.view_as(pred))
        for j in range(len(label)):
                        target = label[j].data
                        train_class_correct[target] += correct[j].item()
                        train_class_total[target] += 1

        running_loss += (loss.item() * img1.size(0))
        train_accuracy = 100. * np.sum(train_class_correct) / np.sum(train_class_total)
        running_train_accuracy += (train_accuracy * img1.size(0))
        
        
        if i%40 == 0:
            print("Epoch number {}\n Current loss {}\n".format(epoch+1,loss.item()))
            
            

            for i in range(num_classes):
                if train_class_total[i] > 0:
                        print('\nTraining Accuracy of %5s: %2d%% (%2d/%2d)' % (
                            str(i), 100 * train_class_correct[i] / train_class_total[i],
                            np.sum(train_class_correct[i]), np.sum(train_class_total[i])))

            print('\nTraining Accuracy (Overall): %2d%% (%2d/%2d)' % (
                100. * np.sum(train_class_correct) / np.sum(train_class_total),
                np.sum(train_class_correct), np.sum(train_class_total)))

    scheduler.step()
    train_loss_history.append(running_loss / len(train_data))
    train_iteration_number +=1
    train_counter.append(train_iteration_number)
    train_accuracy_history.append(running_train_accuracy / len(train_data))
    net.eval()
    for i, data in enumerate(test_loader,0):
     
        img1 , label = data
        img1 , label = img1.cuda(), label.cuda()
        
        output1= model1(img1)
        output = net(output1)
        _, pred= output.max(1)
        
        
        loss = criterion(output,label)
        
        correct = pred.eq(label.view_as(pred))
        for j in range(len(label)):
                        target = label[j].data
                        valid_class_correct[target] += correct[j].item()
                        valid_class_total[target] += 1
        
        running_valid_loss += (loss.item() * img1.size(0))
        val_accuracy = 100. * np.sum(valid_class_correct) / np.sum(valid_class_total)
        running_val_accuracy += (val_accuracy * img1.size(0))               
        if i%50 == 0:
            print("Epoch number {}\n Current loss {}\n".format(epoch+1,loss.item()))



        
            for i in range(num_classes):
                if train_class_total[i] > 0:
                        if valid_class_total[i] == 0:
                            valid_class_total[i] = valid_class_total[i]+0.000001
                        print('\nValidation Accuracy of %5s: %2d%% (%2d/%2d)' % (
                            str(i), 100 * valid_class_correct[i] / (valid_class_total[i]),
                            np.sum(valid_class_correct[i]), np.sum(valid_class_total[i])))

            print('\nValidation Accuracy (Overall): %2d%% (%2d/%2d)' % (
                100. * np.sum(valid_class_correct) / np.sum(valid_class_total),
                np.sum(valid_class_correct), np.sum(valid_class_total)))
            
    
    valid_loss_history.append(running_valid_loss / len(test_data))
    valid_iteration_number +=1
    valid_counter.append(valid_iteration_number)
    val = running_valid_loss / len(test_data) 
    print("Validation loss for epoch {}: {:.6f}".format(epoch+1,val))
    valid_accuracy_history.append(running_val_accuracy / len(test_data))
    if val <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        val))
        torch.save(net, "/content/gdrive/My Drive/pretrained.pth")
        valid_loss_min = val

Epoch number 1
 Current loss 1.6358754634857178


Training Accuracy of     0: 26% ( 7/26)

Training Accuracy of     1:  0% ( 0/ 5)

Training Accuracy of     2:  2% ( 1/35)

Training Accuracy of     3: 87% (42/48)

Training Accuracy of     4: 43% (17/39)

Training Accuracy of     5: 28% (10/35)

Training Accuracy of     6: 22% ( 8/36)

Training Accuracy (Overall): 37% (85/224)
Epoch number 1
 Current loss 1.6674537658691406


Training Accuracy of     0: 16% (204/1253)

Training Accuracy of     1:  0% ( 0/140)

Training Accuracy of     2:  8% (114/1336)

Training Accuracy of     3: 74% (1744/2337)

Training Accuracy of     4: 28% (457/1587)

Training Accuracy of     5: 32% (496/1525)

Training Accuracy of     6: 18% (191/1006)

Training Accuracy (Overall): 34% (3206/9184)
Epoch number 1
 Current loss 1.6075643301010132


Training Accuracy of     0: 18% (475/2550)

Training Accuracy of     1:  0% ( 0/267)

Training Accuracy of     2:  7% (185/2611)

Training Accuracy of     3: 73% (3344/4

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch number 2
 Current loss 1.6304913759231567


Training Accuracy of     0: 12% ( 3/24)

Training Accuracy of     1:  0% ( 0/ 2)

Training Accuracy of     2:  3% ( 1/32)

Training Accuracy of     3: 71% (42/59)

Training Accuracy of     4: 25% (12/47)

Training Accuracy of     5: 41% (14/34)

Training Accuracy of     6: 15% ( 4/26)

Training Accuracy (Overall): 33% (76/224)
Epoch number 2
 Current loss 1.6563359498977661


Training Accuracy of     0: 17% (220/1263)

Training Accuracy of     1:  0% ( 0/144)

Training Accuracy of     2:  6% (84/1368)

Training Accuracy of     3: 71% (1603/2252)

Training Accuracy of     4: 32% (525/1601)

Training Accuracy of     5: 37% (598/1590)

Training Accuracy of     6: 16% (155/966)

Training Accuracy (Overall): 34% (3185/9184)
Epoch number 2
 Current loss 1.659454584121704


Training Accuracy of     0: 17% (427/2494)

Training Accuracy of     1:  0% ( 0/293)

Training Accuracy of     2:  6% (167/2667)

Training Accuracy of     3: 74% (3393/4562

In [0]:
print('\n Loss History')
plt.plot(train_counter,train_loss_history, label="train")
plt.plot(valid_counter,valid_loss_history, label="valid")
plt.legend()
plt.show()


print('\n Accuracy History')
plt.plot(train_counter,train_accuracy_history, label="train")
plt.plot(valid_counter,valid_accuracy_history, label="valid")
plt.legend()
plt.show()



 Loss History


NameError: ignored

In [0]:
net.eval()
model1.eval()

train_counter = []
train_iteration_number= 0

valid_counter = []
valid_iteration_number= 0

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

train_class_correct = list(0 for i in range(num_classes))
train_class_total = list(0 for i in range(num_classes))

valid_class_correct = list(0 for i in range(num_classes))
valid_class_total = list(0 for i in range(num_classes))


for i, data in enumerate(train_loader,0):
        
  img1 , label = data
        #img1, label = Variable(img1), Variable(label)
  img1 , label = img1.cuda(), label.cuda()
        #img1, label = Variable(img1), Variable(label)
        #row, img0, img1 , label = row.cuda(), img0.cuda(), img1.cuda() , label.cuda()
  

  output1= model1(img1)      
  output = net(output1)
  _, pred= torch.max(output,1)
        
        #plt.imshow(img1.data[0])

        #a = plt.imshow(img1.data[0].permute(1, 2, 0))
        #plt.title(train_loader.dataset.classes[pred[0]])
        #a.set_title = train_loader.dataset.classes[pred[0]]
        #plt.show()
  correct = pred.eq(label.view_as(pred))
  for j in range(len(label)):
    target = label[j].data
    train_class_correct[target] += correct[j].item()
    train_class_total[target] += 1

for i in range(num_classes):
    if train_class_total[i] > 0:
        print('\nTraining Accuracy of %5s: %2d%% (%2d/%2d)' % (str(i), 100 * train_class_correct[i] / train_class_total[i],
                                                               np.sum(train_class_correct[i]), np.sum(train_class_total[i])))

print('\nTraining Accuracy (Overall): %2d%% (%2d/%2d)' % (100. * np.sum(train_class_correct) / np.sum(train_class_total),
                                                                  np.sum(train_class_correct), np.sum(train_class_total)))
    
for i, data in enumerate(test_loader,0):
     
  img1 , label = data
  img1 , label = img1.cuda(), label.cuda()
        #row, img0, img1 , label = row.cuda(), img0.cuda(), img1.cuda() , label.cuda()
        
  output1= model1(img1)
  output = net(output1)
        #combined = torch.cat([vgg,res1,sen1],1)
        #output= net(combined)
  _, pred= output.max(1)
        
        
       
  correct = pred.eq(label.view_as(pred))
  for j in range(len(label)):
    target = label[j].data
    valid_class_correct[target] += correct[j].item()
    valid_class_total[target] += 1
                       

for i in range(num_classes):
    if train_class_total[i] > 0:
          print('\nValdiation Accuracy of %5s: %2d%% (%2d/%2d)' % (str(i), 100 * valid_class_correct[i] / valid_class_total[i],
                                                                   np.sum(valid_class_correct[i]), np.sum(valid_class_total[i])))

print('\nValidation Accuracy (Overall): %2d%% (%2d/%2d)' % (100. * np.sum(valid_class_correct) / np.sum(valid_class_total),
                                                                      np.sum(valid_class_correct), np.sum(valid_class_total)))